____________________

In [136]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import  f1_score
from utils import load_obj

In [35]:
# Путь к файлу с мета данными
meta_pth = '/media/grigory/Диск/ITMO_DATA/data_v_7_stc/meta/meta.txt'

# Пути к директориям с аудиофайлами.
train_audio_pth = '/media/grigory/Диск/ITMO_DATA/data_v_7_stc/audio'
test_audio_pth = '/media/grigory/Диск/ITMO_DATA/data_v_7_stc/test'

extracted_data = 'data/extracted'

# Имена файлов с извлечёнными признаками для dense сети, линейных моделей и деревьев.
extracted_train = 'features_labels_train' 
extracted_test = 'features_labels_test'

загружаем извлечённые признаки

In [36]:
X, names, labels = load_obj(os.path.join(extracted_data, extracted_train))
X_test, names_test = load_obj(os.path.join(extracted_data, extracted_test))[:2]

In [37]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# кодируем таргет
le = LabelEncoder()
y = le.fit_transform(labels)
ohe =  OneHotEncoder()
y_ohe = ohe.fit_transform(y.reshape(-1,1))

In [38]:
n_features = X.shape[1]
n_classes = y.max()+1
print('Число классов: {}\nЧисло признаков: {}\nЧисло сэмплов: {}'.format(n_classes, n_features, y.shape[0]))

Число классов: 8
Число признаков: 193
Число сэмплов: 11307


> для правильной валидации нужно учитывать, что некоторые сэмплы являются частями одной записи.
Если части одной записи будут разрознено находится и в train и в val, то произойдёт утечка меток в валидацию.

Поэтому сперва найдём уникальные записи и сгруппируем все фрагменты по принадлежности к отдельной записи.

In [39]:
groups_le = LabelEncoder()
unique_samples = pd.Series(names).apply(lambda x: x.split('time_stretch')[0].strip('.wav').strip('_'))
groups = groups_le.fit_transform(unique_samples)

In [40]:
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, train_size=0.8, random_state=7)

idxs = np.arange(X.shape[0])
tr_idxs, val_idxs = next(iter(gss.split(idxs, groups=groups)))

In [41]:
# сплитим...
# имена файлов
names_train, names_val = names[tr_idxs], names[val_idxs]
# извлечённые признаки
X_train, X_val = X[tr_idxs], X[val_idxs]
# метки для sklearn
y_train, y_val = y[tr_idxs], y[val_idxs]
# метки для сеток
y_ohe_train, y_ohe_val = y_ohe[tr_idxs], y_ohe[val_idxs]

**Update:**
ниже некорректный сплит

In [42]:
# # сплитим
# tmp = train_test_split(names, X, y, y_ohe, train_size=0.8, test_size=0.2, stratify=y)

# # имена файлов, входные данные, метки для sklearn, метки для сеток
# names_train, names_val, X_train, X_val, y_train, y_val, y_ohe_train, y_ohe_val = tmp

### Линейные модели

In [44]:
# from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
# C_grid = np.logspace(-3,4,10) 
# lr = LogisticRegressionCV(C_grid, n_jobs=-1) #LogisticRegression()

In [45]:
# %%time
# # Обучаем
# print('Training...')
# lr.fit(X_train, y_train)

Training...
CPU times: user 10.1 s, sys: 572 ms, total: 10.7 s
Wall time: 1min 12s


In [46]:
# # Оцениваем качества на отложенной выборке
# print('Scoring...')
# tr_score = lr.score(X_train, y_train)
# te_score = lr.score(X_val, y_val)
# print('Acc:: Train score: {}, test score: {}'.format(tr_score, te_score))

# tr_score = f1_score(y_train, lr.predict(X_train), average='weighted')
# te_score = f1_score(y_val, lr.predict(X_val), average='weighted')
# print('F1:: Train score: {}, test score: {}'.format(tr_score, te_score))

Scoring...
Acc:: Train score: 0.91414029652578, test score: 0.879682679594535
F1:: Train score: 0.913935660013014, test score: 0.8805494911071801


## Нелинейные модели

#### бустинг и лес

In [164]:
# # %%time
# # !pip install lightgbm
# import lightgbm as lgb
# from lightgbm.sklearn import LGBMClassifier

# trees = LGBMClassifier(learning_rate=1e-2,
# #                        reg_alpha=1e-2,
# #                        reg_beta=1e-1,
# #                      valid_sets=[X_val, y_val],
#                        random_state=7,
#                      n_estimators=500)
# trees.fit(X_train, y_train);

In [165]:
# %%time
# from sklearn.ensemble import RandomForestClassifier
# trees = RandomForestClassifier(n_estimators=100, max_depth=25, max_features=120)

# trees.fit(X_train, y_train)

In [167]:
# # Оцениваем качества на отложенной выборке
# print('Scoring...')
# tr_score = trees.score(X_train, y_train)
# te_score = trees.score(X_val, y_val)
# print('Acc:: Train score: {}, test score: {}'.format(tr_score, te_score))

Scoring...
Acc:: Train score: 1.0, test score: 0.9471132657558395


переобучились немножк...

#### онлайн-обучение нейонных сеток

Обзор методов на нейронных сетях:

- http://www.fim.uni-passau.de/fileadmin/files/lehrstuhl/schuller/Publications/Amiriparian17-SSC.pdf
- https://github.com/libphy/which_animal
- https://github.com/jaron/deep-listening
- https://musicinformationretrieval.com/mfcc.html

Тестировались:
    - полносвязанные сети
    - lstm
    - свёрточные

Самые лучшие результаты показали полносвязные сети.


*Коротко о подборе оптимальной архитектуры для Dense сети*:

<img width=350 src="https://habrastorage.org/webt/sg/7t/tu/sg7ttuirleaml3_j7dwo2tn0iqs.png">

In [149]:
import keras
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [158]:
from models import (build_deep_dense, build_conv_seq, build_lstm_seq)
K.clear_session()

##### Dense сеть

In [159]:
model = build_deep_dense(n_features, n_classes)


model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(1e-4), 
              metrics=['accuracy'])
callbacks = [
    EarlyStopping(patience=25, monitor='val_loss'),
    ModelCheckpoint('models_weights/net_ff.h5', monitor='val_loss', 
                    verbose=1, save_best_only=True, period=4)
]
model.fit(X_train, y_ohe_train.todense(), 
          epochs=300, batch_size=250, 
          verbose=0,
          validation_data=[X_val, y_ohe_val.todense()],
         callbacks=callbacks)

Epoch 00004: val_loss improved from inf to 1.01507, saving model to models_weights/net_ff.h5
Epoch 00008: val_loss improved from 1.01507 to 0.64368, saving model to models_weights/net_ff.h5
Epoch 00012: val_loss improved from 0.64368 to 0.48766, saving model to models_weights/net_ff.h5
Epoch 00016: val_loss improved from 0.48766 to 0.41567, saving model to models_weights/net_ff.h5
Epoch 00020: val_loss improved from 0.41567 to 0.35174, saving model to models_weights/net_ff.h5
Epoch 00024: val_loss improved from 0.35174 to 0.33874, saving model to models_weights/net_ff.h5
Epoch 00028: val_loss improved from 0.33874 to 0.30450, saving model to models_weights/net_ff.h5
Epoch 00032: val_loss did not improve
Epoch 00036: val_loss improved from 0.30450 to 0.25036, saving model to models_weights/net_ff.h5
Epoch 00040: val_loss did not improve
Epoch 00044: val_loss did not improve
Epoch 00048: val_loss improved from 0.25036 to 0.22810, saving model to models_weights/net_ff.h5
Epoch 00052: val_

##### Рекурретная сеть

In [32]:
# model = build_lstm_seq(timesteps=20, data_dim=41, n_classes=n_classes)
# model.compile(loss='categorical_crossentropy', 
#               optimizer=Adam(1e-4), 
#               metrics=['accuracy'])
# callbacks = [
#     EarlyStopping(patience=25, monitor='val_loss'),
#     ModelCheckpoint('models_weights/net_lstm.h5', monitor='val_loss', 
#                     verbose=1, save_best_only=True, period=4)
# ]

# model.fit(X_train, y_ohe_train.todense(), 
#           epochs=300, batch_size=250, 
#           validation_data=[X_val, y_ohe_val.todense()],
#          callbacks=callbacks)

Train on 92491 samples, validate on 21060 samples
Epoch 1/300
92491/92491 [==============================] - 51s 549us/step - loss: 0.8721 - acc: 0.7172 - val_loss: 0.4160 - val_acc: 0.8614
Epoch 2/300
92491/92491 [==============================] - 49s 528us/step - loss: 0.3717 - acc: 0.8784 - val_loss: 0.3155 - val_acc: 0.8955
Epoch 3/300
92491/92491 [==============================] - 50s 536us/step - loss: 0.3038 - acc: 0.9006 - val_loss: 0.2986 - val_acc: 0.9009
Epoch 4/300
92491/92491 [==============================] - 49s 531us/step - loss: 0.2630 - acc: 0.9143 - val_loss: 0.2793 - val_acc: 0.9088
Epoch 5/300
92491/92491 [==============================] - 49s 530us/step - loss: 0.2353 - acc: 0.9231 - val_loss: 0.2605 - val_acc: 0.9162
Epoch 6/300
92491/92491 [==============================] - 49s 529us/step - loss: 0.2162 - acc: 0.9296 - val_loss: 0.2564 - val_acc: 0.9183
Epoch 7/300
92491/92491 [==============================] - 49s 530us/step - loss: 0.2020 - acc: 0.9331 - val_l

###

In [160]:
from keras.models import load_model 
model = load_model('models_weights/net_ff.h5')

In [161]:
from utils import inverse_ohe
preds = model.predict(X_train)
preds_labels = inverse_ohe(preds, ohe)
tr_score = f1_score(y_train, preds_labels, average='weighted')
preds = model.predict(X_val)
preds_labels = inverse_ohe(preds, ohe)
te_score = f1_score(y_val, preds_labels, average='weighted')
print('F1:: Train score: {}, val score: {}'.format(tr_score, te_score))

F1:: Train score: 0.9848220389975106, val score: 0.9530651061049963


Выводы по моделям:

> Наилучшими моделями(без яростного тюнинга параметров) оказались полносвязная сеть и бустинг над решающими деревьями. Они дают  сопоставимые результаты.


### Предикт теста и подготовка сабмита

In [162]:
preds = model.predict(X_test)

In [163]:
def prepare_submit(names, preds, ohe=ohe, le=le):
    pred_labels = inverse_ohe(preds, ohe)
    pred_names = le.inverse_transform(pred_labels)
    df = pd.DataFrame(list(zip(names, preds.max(axis=1), pred_names)), columns=['file','prob','label'])
    return df
                      
submit_df = prepare_submit(names_test, preds)
submit_df.to_csv('result.txt', sep='\t', header=None, index=None)